In [1]:
import math

For this problem, we need to identify $$\begin{pmatrix} 10^{18} \\ 10^9 \end{pmatrix} \mod p$$ for an aribtary prime $p$ in that range $[1001, 4999]$ using [Lucas's theorem](https://en.wikipedia.org/wiki/Lucas%27s_theorem#Statement)

First we need to write a number $m$ in base $p$

In [2]:
def expand_base(n: int, p: int) -> list[int]:
    l = []
    while n > 0:
        l.append(n % p)
        n //= p
    return l[::-1]

expand_base(101, 100)

[1, 1]

We generate the set of factorials from $0$ to $5000$, and use it for modulos via bigint-mod-smallint scenario. I'm smart

In [3]:
# since this is Python it allows us to do exceptionally stupid things
import sys
sys.set_int_max_str_digits(0)

fac = ["1"]
for i in range(1, 5001):
    fac.append(str(int(fac[-1]) * i))
print(fac[10])

3628800


In [4]:
def largeint_mod(s: str, p: int) -> int:
    t = 0
    for i in s:
        t = (10 * t + int(i)) % p
    return t % p

largeint_mod(fac[100], 5000)

0

In [5]:
def factorial_mod(n: int, p: int) -> int:
    global fac
    # we can optimize this a bit
    if n >= p: return 0
    return largeint_mod(fac[n], p)

def factorial_mod_inverse(n: int, p: int) -> int:
    global fac
    a = factorial_mod(n, p)
    if math.gcd(a, p) > 1: return -1 # fun fact gcd handles the case a = 0 too
    else: return pow(a, p - 2, p)
    
factorial_mod(6, 7)

6

In [6]:
def binomial_coefficient_mod(m: int, n: int, p: int) -> int:
    v = expand_base(m, p)
    z = expand_base(n, p)
    while len(v) < len(z):
        v = [0] + v
    while len(z) < len(v):
        z = [0] + z
    
    t = 1
    for x, y in zip(v, z):
        if x < y: 
            t = 0
            break
        t = (t * factorial_mod(x, p)) % p
        t = (t * factorial_mod_inverse(y, p)) % p
        t = (t * factorial_mod_inverse(x - y, p)) % p
    return t

math.comb(1124, 14) % (419)

348

In [7]:
binomial_coefficient_mod(1124, 14, 419)

348

ok so we just take this function to steal primes

In [17]:
def small_miller_rabin(n: int) -> bool:
    p = [2, 3, 5, 7, 11, 13, 17, 19]
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    for i in p:
        if n == i:
            return True
        elif n % i == 0:
            return False
        a = i
        x = pow(a, d, n)
        for _ in range(s):
            y = pow(x, 2, n)
            if y == 1 and x != 1 and x != n - 1:
                return False
            x = y
        if y != 1:
            return False
    return True


primes = [i for i in range(1001, 5000) if small_miller_rabin(i)]
print(primes, len(primes))

[1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 1223, 1229, 1231, 1237, 1249, 1259, 1277, 1279, 1283, 1289, 1291, 1297, 1301, 1303, 1307, 1319, 1321, 1327, 1361, 1367, 1373, 1381, 1399, 1409, 1423, 1427, 1429, 1433, 1439, 1447, 1451, 1453, 1459, 1471, 1481, 1483, 1487, 1489, 1493, 1499, 1511, 1523, 1531, 1543, 1549, 1553, 1559, 1567, 1571, 1579, 1583, 1597, 1601, 1607, 1609, 1613, 1619, 1621, 1627, 1637, 1657, 1663, 1667, 1669, 1693, 1697, 1699, 1709, 1721, 1723, 1733, 1741, 1747, 1753, 1759, 1777, 1783, 1787, 1789, 1801, 1811, 1823, 1831, 1847, 1861, 1867, 1871, 1873, 1877, 1879, 1889, 1901, 1907, 1913, 1931, 1933, 1949, 1951, 1973, 1979, 1987, 1993, 1997, 1999, 2003, 2011, 2017, 2027, 2029, 2039, 2053, 2063, 2069, 2081, 2083, 2087, 2089, 2099, 2111, 2113, 2129, 2131, 2137, 2141, 2143, 2153, 2161, 2179, 2203, 2207, 2213, 2221, 2237, 2239, 2243, 225

In [15]:
a = 2084212
b = 41
for i in primes:
    if math.comb(a, b) % i != binomial_coefficient_mod(a, b, i):
        print(i)
        break

ok so our binomial mod function is probably legit
let's construct the big mod 

In [18]:
p_mod = {i: binomial_coefficient_mod(10 ** 18, 10 ** 9, i) for i in primes}

In [11]:
list(p_mod.values()).count(0)

376

In [12]:
# would be better to code this seperately
def modular_inverse(a: int, p: int) -> int:
    return pow(a, p - 2, p)

def crt(values: list[tuple[int, int]]) -> tuple[int, int]:
    # i[0] are the modulus, i[1] is the prime itself
    n = len(values)
    l = [i[0] for i in values]
    p = [i[1] for i in values]
    M = [1 for i in range(n)]
    
    m = 1
    for i in p:
        m *= i
    M = [m // i for i in p]
    N = [modular_inverse(M[i], p[i]) for i in range(n)]
    t = 0
    m = 1
    for i, x, y in zip(values, M, N):
        t += i[0] * x * y
        m *= i[1]
    return (t, m)

# test: a mod 2 = 0, a mod 3 = 1, a mod 5 = 1
crt([(0, 2), (1, 3), (1, 5), (1, 7)])

(316, 210)

In [13]:
import itertools
t = 0

# def modular_inverse(a: int, p: int) -> int:
#     return pow(a % p, p - 2, p)

for i in itertools.combinations(primes, 3):
    p, q, r = i[0], i[1], i[2]
    # CHINESE
    t += crt([
        (p_mod[i], i) for i in [p, q, r]
    ])[0]
    
print(t)

397836820380158570970


In [14]:
for a in zip([1, 2, 3], [4, 5, 6], [7, 8 ,9]):
    print(a)

(1, 4, 7)
(2, 5, 8)
(3, 6, 9)
